In [2]:
import os
import sys
from pathlib import Path

# jupyter/ 폴더 내의 파일에서 api/ 폴더를 path에 추가하여 import 할 수 있도록 하는 코드
current_dir = Path(os.path.abspath(''))
parent_dir = current_dir.parent
if str(parent_dir) not in sys.path:
    sys.path.insert(0, str(parent_dir))

from api.module.sqlitevec import SQLiteVecDB
from api.module.embedding import Embedding
from api.module.utils import get_db_filename

c:\Users\psdae\AppData\Local\miniconda3\envs\rag-db-adam\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd
from tqdm import tqdm

### 1) 데이터 전처리

- 채용공고 csv파일을 불러와서 df 데이터프레임에 `query`열과 `content`열로 구성되도록 데이터를 처리합니다

In [4]:
raw = pd.read_csv("datas/채용공고_example.csv")
print(raw.columns)

columns = ['기업명', '모집공고', '모집 부서', '구분', '경력', '연도', '시작일자',
       '마감일자', '업무', '자격요건', '우대사항', '금융기업', '데이터직무', '코딩테스트', '기술스택',
       '기타, 참고사항']

df = raw[columns].copy()
df.info()
df

Index(['Created', 'By', '기업명', '모집공고', '모집 부서', '구분', '경력', '연도', '시작일자',
       '마감일자', '업무', '자격요건', '우대사항', '금융기업', '데이터직무', '코딩테스트', '기술스택',
       '기타, 참고사항', '태그_데이터 관련 자격증', '태그_학력', '페이지 링크', '태그_개발역량 (삭제 예정)',
       '태그, 키워드 (삭제 예정)'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   기업명       22 non-null     object 
 1   모집공고      22 non-null     object 
 2   모집 부서     8 non-null      object 
 3   구분        22 non-null     object 
 4   경력        22 non-null     object 
 5   연도        22 non-null     int64  
 6   시작일자      20 non-null     object 
 7   마감일자      14 non-null     object 
 8   업무        22 non-null     object 
 9   자격요건      20 non-null     object 
 10  우대사항      22 non-null     object 
 11  금융기업      22 non-null     object 
 12  데이터직무     22 non-null     object 
 13  코딩테스트     0 non-null      float64
 14  

,기업명,모집공고,모집 부서,구분,경력,연도,시작일자,마감일자,업무,자격요건,우대사항,금융기업,데이터직무,코딩테스트,기술스택,"기타, 참고사항"
0,(주)네피리티,[(주)네피리티/서울] 데이터 분석 및 AI모델 개발 모집,NaN,"계약직/파견직, 정규직","경력(연차 무관), 신입",2025,2025/02/14,2025/03/16,- 데이터 수집 및 정제\n- 데이터 분석 및 시각화\n- AI/머신러닝 모델 개발...,- 경력: 신입 및 경력무관\n- 학력: 학사이상\n- 배움의 대한 열정과 책임감이...,"- 컴퓨터/시스템공학, 공학계열\n- 정보처리기사\n- 해당직무 근무경험",No,Yes,NaN,NaN,NaN
1,(주)마크클라우드,"Al 개발(Python), 데이터분석 및 사업 기획 모집",AI 팀,정규직(수습),"5년 이하, 신입",2025,2025/01/21,NaN,"Big Data, 통계, 분석, 모델링, 시각화","ㆍ데이터 분석을 위한 프로그래밍 역량 (Python, R, SQL 등)\nㆍ데이터 ...",ㆍ정부 또는 기관에서 진행하는 빅데이터 분석과정을 수료하거나 관련 공모전에서 수상한...,No,Yes,NaN,"Python, R, SQL, Pandas, Numpy, NLP","수습기간 3개월, 채용시 마감"
2,(주)마크클라우드,"Al 개발(Python), 데이터분석 및 사업 기획 모집",AI 팀,정규직(수습),"5년 이하, 신입",2025,2025/01/21,NaN,AI 모델 연구 및 개발(Python),"ㆍ이미지, 텍스트 머신러닝/딥러닝 엔진 개선 및 Fine-tuning\nㆍPytor...",ㆍ정부 또는 기관에서 진행하는 빅데이터 분석과정을 수료하거나 관련 공모전에서 수상한...,No,No,NaN,"Python, Pytorch, Keras, Tensorflow","수습기간 3개월, 채용시 마감"
3,(주)에버스핀,[신입/경력] 빅데이터 분석 마케터 채용,NaN,정규직(수습),"경력 무관, 신입",2025,2025/01/31,2025/03/02,- 모바일 빅데이터 분석을 통해 금융 사기와 관련된 인사이트 발굴\n- 분석된 데이...,- 전문대졸 이상\n- 신입 가능(데이터 분석 관련 학과 혹은 관련 자격 취득자),"- 엑셀, 워드 등 OA 중급이상 사용가능\n- SQL 사용가능 (SELECT, J...",No,Yes,NaN,SQL,데이터 분석 관련 학과 혹은 관련 자
4,(주)바이오바이츠,헬스케어 인공지능 모델 개발 및 데이터 분석 PM 채용,NaN,정규직(수습),"경력 무관, 신입",2025,2025/01/23,NaN,[주요업무]\n- 헬스케어 분야의 인공지능 모델 개발을 담당합니다.\n- 데이터 분...,"[자격요건]\n- 컴퓨터 공학, 데이터 과학 또는 관련 분야의 학사 학위 이상 소...",[우대사항]\n- 의료 데이터 분석 경험이 있으신 분 \n- 헬스케어 분야의 인공...,No,Yes,NaN,"Python, R, SQL","수습기간 3개월, 채용시 마감"
5,디큐월드(유),주니어 데이터 사이언티스트 채용 공고,NaN,정규직,신입,2025,2025/02/03,NaN,"주니어 데이터 사이언티스트는 DQ Lab의 데이터 팀에서 데이터 수집, 정제, 분석...","필수 요건\n- 컴퓨터 과학, 데이터 과학, 통계학, 수학 또는 관련 분야 학사 학...",다음과 같은 경험이 있다면 더욱 좋습니다.\n- 영어 커뮤니케이션 능력 (글로벌 팀...,No,Yes,NaN,"Python, R, SQL",채용시 마감
6,메가존클라우드(주),데이터 분석가 채용,NaN,정규직(수습),"경력 무관, 신입",2025,2025/02/20,2025/04/21,- Cloud(AWS) Billing 데이터 분석\n- RI/SP 운영 업무\n- ...,- 데이터 분석에 관심이 많은 분\n- 엑셀/스프레드시트 활용 능숙한 분\n- 숫자...,"- SQL Query 활용 경험 (Athena, BIgquery, Databrick...",No,Yes,NaN,NaN,수습기간 3개월
7,(주)대연,[NIKE PARTNER] 데이터 사이언티스트 채용,NaN,계약직/파견직,"경력 무관, 신입",2025,2025/02/14,2025/03/16,"- 데이터 분석 및 모델링: 판매, 고객, 제고 등 데이터 분석\n- 경영 전력 수...",- 경력: 무관\n- 학력: 무관\n- Python을 활용한 데이터 분석 및 프로그...,- 비즈니스에 대한 높은 이해도\n- 팀원 및 타 부서와의 원활한 커뮤니케이션\n-...,No,Yes,NaN,NaN,계약직 1년/정규직 전환 가능
8,(주)씨아이템프러리,현대카드 카드기획팀 데이터 추출 및 분석가 채용 /SQL/,카드기획팀,계약직/파견직,"경력 무관, 신입",2025,2025/01/24,2025/02/23,- SQL을 활용한 실적 데이터 추출 및 분석\n- 회원 카드 사용/가맹점 매출 및...,NaN,*데이터 추출/분석 업무 경력자\n*SQL 관련 자격증 소지자\n*데이터 관련 전공...,No,Yes,NaN,NaN,NaN
9,(주)사람인에이치에스,[워라밸/신입가능] 한국정보통신진흥협회 AI데이터사업팀 사무직,AI 데이터 사업팀,계약직/파견직,"경력 무관, 신입",2025,2025/02/17,2025/02/24,행정 사무 업무,NaN,컴퓨터 활용 가능자 (excel),No,No,NaN,NaN,고용형태: 사람인HS 소속 파견계약직 (입사일 ~ 2년 근무)\n *한국정보통신진흥...


In [5]:
rag_df = pd.DataFrame(columns=["query", "content"])

# content 내용에 포함할 컬럼
content_cols = ["모집공고", "업무", "모집 부서", "구분", "경력", "자격요건", "우대사항", "기술스택", "기타, 참고사항"]

rows = []
for i in range(len(df)):
    # content
    content_strings = [f"{col}: {df[col][i]}" for col in content_cols]
    content = "\n---\n".join(content_strings)

    # query
    query = content
    # NOTE: 원래는 query를 따로 구성해야 하나, 현재 테스트 단계에서는 content를 그대로 사용함

    row = {"query": query, "content": content}
    rows.append(row)

rag_df = pd.DataFrame(rows)
rag_df

,query,content
0,모집공고: [(주)네피리티/서울] 데이터 분석 및 AI모델 개발 모집\n---\n업...,모집공고: [(주)네피리티/서울] 데이터 분석 및 AI모델 개발 모집\n---\n업...
1,"모집공고: Al 개발(Python), 데이터분석 및 사업 기획 모집\n---\n업무...","모집공고: Al 개발(Python), 데이터분석 및 사업 기획 모집\n---\n업무..."
2,"모집공고: Al 개발(Python), 데이터분석 및 사업 기획 모집\n---\n업무...","모집공고: Al 개발(Python), 데이터분석 및 사업 기획 모집\n---\n업무..."
3,모집공고: [신입/경력] 빅데이터 분석 마케터 채용\n---\n업무: - 모바일 빅...,모집공고: [신입/경력] 빅데이터 분석 마케터 채용\n---\n업무: - 모바일 빅...
4,모집공고: 헬스케어 인공지능 모델 개발 및 데이터 분석 PM 채용\n---\n업무:...,모집공고: 헬스케어 인공지능 모델 개발 및 데이터 분석 PM 채용\n---\n업무:...
5,모집공고: 주니어 데이터 사이언티스트 채용 공고\n---\n업무: 주니어 데이터 사...,모집공고: 주니어 데이터 사이언티스트 채용 공고\n---\n업무: 주니어 데이터 사...
6,모집공고: 데이터 분석가 채용\n---\n업무: - Cloud(AWS) Billin...,모집공고: 데이터 분석가 채용\n---\n업무: - Cloud(AWS) Billin...
7,모집공고: [NIKE PARTNER] 데이터 사이언티스트 채용\n---\n업무: -...,모집공고: [NIKE PARTNER] 데이터 사이언티스트 채용\n---\n업무: -...
8,모집공고: 현대카드 카드기획팀 데이터 추출 및 분석가 채용 /SQL/\n---\n업...,모집공고: 현대카드 카드기획팀 데이터 추출 및 분석가 채용 /SQL/\n---\n업...
9,모집공고: [워라밸/신입가능] 한국정보통신진흥협회 AI데이터사업팀 사무직\n---\...,모집공고: [워라밸/신입가능] 한국정보통신진흥협회 AI데이터사업팀 사무직\n---\...


In [6]:
# rag db 생성용 데이터프레임
rag_df.info()
rag_df.to_csv("temp/채용공고_example_df.csv")
rag_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   query    22 non-null     object
 1   content  22 non-null     object
dtypes: object(2)
memory usage: 484.0+ bytes


,query,content
0,모집공고: [(주)네피리티/서울] 데이터 분석 및 AI모델 개발 모집\n---\n업...,모집공고: [(주)네피리티/서울] 데이터 분석 및 AI모델 개발 모집\n---\n업...
1,"모집공고: Al 개발(Python), 데이터분석 및 사업 기획 모집\n---\n업무...","모집공고: Al 개발(Python), 데이터분석 및 사업 기획 모집\n---\n업무..."
2,"모집공고: Al 개발(Python), 데이터분석 및 사업 기획 모집\n---\n업무...","모집공고: Al 개발(Python), 데이터분석 및 사업 기획 모집\n---\n업무..."
3,모집공고: [신입/경력] 빅데이터 분석 마케터 채용\n---\n업무: - 모바일 빅...,모집공고: [신입/경력] 빅데이터 분석 마케터 채용\n---\n업무: - 모바일 빅...
4,모집공고: 헬스케어 인공지능 모델 개발 및 데이터 분석 PM 채용\n---\n업무:...,모집공고: 헬스케어 인공지능 모델 개발 및 데이터 분석 PM 채용\n---\n업무:...
5,모집공고: 주니어 데이터 사이언티스트 채용 공고\n---\n업무: 주니어 데이터 사...,모집공고: 주니어 데이터 사이언티스트 채용 공고\n---\n업무: 주니어 데이터 사...
6,모집공고: 데이터 분석가 채용\n---\n업무: - Cloud(AWS) Billin...,모집공고: 데이터 분석가 채용\n---\n업무: - Cloud(AWS) Billin...
7,모집공고: [NIKE PARTNER] 데이터 사이언티스트 채용\n---\n업무: -...,모집공고: [NIKE PARTNER] 데이터 사이언티스트 채용\n---\n업무: -...
8,모집공고: 현대카드 카드기획팀 데이터 추출 및 분석가 채용 /SQL/\n---\n업...,모집공고: 현대카드 카드기획팀 데이터 추출 및 분석가 채용 /SQL/\n---\n업...
9,모집공고: [워라밸/신입가능] 한국정보통신진흥협회 AI데이터사업팀 사무직\n---\...,모집공고: [워라밸/신입가능] 한국정보통신진흥협회 AI데이터사업팀 사무직\n---\...


### 2. RAG DB 생성

- rag_df를 통해 RAG DB를 생성합니다

In [7]:
# load_env.py를 먼저 실행 후 아래 코드를 실행하세요
api_key = os.environ["LLM_API_KEY"]
source = os.environ["EMBEDDING_SOURCE"]
model = os.environ["EMBEDDING_MODEL"]
print(api_key[:5]+"...", source, model)

AIzaS... google models/text-embedding-004


In [8]:
# 임베딩 클래스 로드
embedding = Embedding(api_key=api_key, source=source, model=model)

In [9]:
# 임베딩 테스트
vec = embedding.embed("테스트")
len(vec)

768

In [ ]:
# VecDB 클래스 로드 (임베딩 함수 지정)
# initialize(초기화)하여 로드(기존 DB 내용 삭제)
db_filename = get_db_filename(source=source, model=model, nrows=len(rag_df), njobs=len(rag_df["content"].unique()), string="테스트 DB")
print(db_filename)
vecdb = SQLiteVecDB(f"temp/{db_filename}", embedding_function=embedding.embed, init_db=True)  

RAGDB_20250226_193132_google(models_text-embedding-004)_nrows22_njobs22_테스트 DB.sqlite


In [11]:
# query를 임베딩하여 vecdb 생성
for i, row in tqdm(rag_df.iterrows(), total=len(rag_df)):
    query = row["query"]
    content = row["content"]
    vecdb.add_one(query=query, content=content)

100%|██████████| 22/22 [00:26<00:00,  1.20s/it]


In [12]:
# 테스트
query = "SQLD 자격증"
result_df = vecdb.search(query, k=5)
result_df

,query,content,distance
order,,,
1,모집공고: 현대카드 카드기획팀 데이터 추출 및 분석가 채용 /SQL/\n---\n업...,모집공고: 현대카드 카드기획팀 데이터 추출 및 분석가 채용 /SQL/\n---\n업...,0.857251
2,모집공고: [신입/경력] 빅데이터 분석 마케터 채용\n---\n업무: - 모바일 빅...,모집공고: [신입/경력] 빅데이터 분석 마케터 채용\n---\n업무: - 모바일 빅...,0.912878
3,모집공고: 주니어 데이터 사이언티스트 채용 공고\n---\n업무: 주니어 데이터 사...,모집공고: 주니어 데이터 사이언티스트 채용 공고\n---\n업무: 주니어 데이터 사...,0.992285
4,모집공고: 데이터 분석가 인턴\n---\n업무: - KPI 모니터링• 업데이트 콘텐...,모집공고: 데이터 분석가 인턴\n---\n업무: - KPI 모니터링• 업데이트 콘텐...,0.999428
5,모집공고: Data Analytics Engineer\n---\n업무: • Data...,모집공고: Data Analytics Engineer\n---\n업무: • Data...,1.002205


In [13]:
vecdb.get_all()

,embedding,query,content,timestamp
0,"[0.037943, 0.04715, -0.058711, 0.00906, 0.0348...",모집공고: [(주)네피리티/서울] 데이터 분석 및 AI모델 개발 모집\n---\n업...,모집공고: [(주)네피리티/서울] 데이터 분석 및 AI모델 개발 모집\n---\n업...,1.740566e+09
1,"[0.027593, 0.012804, -0.065737, 0.025923, 0.07...","모집공고: Al 개발(Python), 데이터분석 및 사업 기획 모집\n---\n업무...","모집공고: Al 개발(Python), 데이터분석 및 사업 기획 모집\n---\n업무...",1.740566e+09
2,"[0.016461, 0.030933, -0.05518, -0.002283, 0.06...","모집공고: Al 개발(Python), 데이터분석 및 사업 기획 모집\n---\n업무...","모집공고: Al 개발(Python), 데이터분석 및 사업 기획 모집\n---\n업무...",1.740566e+09
3,"[0.048645, 0.053602, -0.034335, 0.002865, 0.05...",모집공고: [신입/경력] 빅데이터 분석 마케터 채용\n---\n업무: - 모바일 빅...,모집공고: [신입/경력] 빅데이터 분석 마케터 채용\n---\n업무: - 모바일 빅...,1.740566e+09
4,"[0.034895, 0.016899, -0.057134, 0.020632, 0.07...",모집공고: 헬스케어 인공지능 모델 개발 및 데이터 분석 PM 채용\n---\n업무:...,모집공고: 헬스케어 인공지능 모델 개발 및 데이터 분석 PM 채용\n---\n업무:...,1.740566e+09
5,"[0.049514, 0.023111, -0.074482, 0.014457, 0.06...",모집공고: 주니어 데이터 사이언티스트 채용 공고\n---\n업무: 주니어 데이터 사...,모집공고: 주니어 데이터 사이언티스트 채용 공고\n---\n업무: 주니어 데이터 사...,1.740566e+09
6,"[0.06327, 0.004594, -0.047706, 0.001944, 0.077...",모집공고: 데이터 분석가 채용\n---\n업무: - Cloud(AWS) Billin...,모집공고: 데이터 분석가 채용\n---\n업무: - Cloud(AWS) Billin...,1.740566e+09
7,"[0.033422, 0.021759, -0.012383, 0.02133, 0.079...",모집공고: [NIKE PARTNER] 데이터 사이언티스트 채용\n---\n업무: -...,모집공고: [NIKE PARTNER] 데이터 사이언티스트 채용\n---\n업무: -...,1.740566e+09
8,"[0.040708, 0.026294, -0.060129, 0.010434, 0.06...",모집공고: 현대카드 카드기획팀 데이터 추출 및 분석가 채용 /SQL/\n---\n업...,모집공고: 현대카드 카드기획팀 데이터 추출 및 분석가 채용 /SQL/\n---\n업...,1.740566e+09
9,"[0.014416, 0.063503, -0.063705, 0.012595, 0.06...",모집공고: [워라밸/신입가능] 한국정보통신진흥협회 AI데이터사업팀 사무직\n---\...,모집공고: [워라밸/신입가능] 한국정보통신진흥협회 AI데이터사업팀 사무직\n---\...,1.740566e+09
